In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup


import warnings

warnings.filterwarnings("ignore")

In [81]:
url = "https://en.wikipedia.org/wiki/Radiohead"

In [82]:
page = requests.get(url)

In [83]:
soup = BeautifulSoup(page.text,"html.parser")
paragraphs = soup.find_all("p")
for paragraph in paragraphs:
    
    text_list = [paragraph.get_text() for paragraph in paragraphs]
print(text_list)
type(text_list)
    

['\n', "Radiohead are an English rock band formed in Abingdon, Oxfordshire, in 1985. They comprise Thom Yorke (vocals, guitar, piano, keyboards); brothers Jonny Greenwood (guitar, keyboards, other instruments) and Colin Greenwood (bass); Ed O'Brien (guitar, backing vocals); and Philip Selway (drums, percussion). They have worked with the producer Nigel Godrich and the cover artist Stanley Donwood since 1994. Radiohead's experimental approach is credited with advancing the sound of alternative rock.\n", 'Radiohead signed to EMI in 1991 and released their debut album, Pablo Honey, in 1993. Their debut single, "Creep", was a worldwide hit, and their popularity and critical standing rose with The Bends in 1995. Their third album, OK Computer (1997), is acclaimed as a landmark record and one of the greatest albums in popular music, with complex production and themes of modern alienation. Their fourth album, Kid A (2000), marked a dramatic change in style, incorporating influences from elect

list

In [ ]:
import spacy
import re
# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")
def preprocess(text_list):
    full_text = ''.join(text_list)
    print(full_text)
    # Process the full_text with spaCy
    doc = nlp(full_text)

    # Filter tokens and convert to lowercase
    filtered_tokens = [token.text.lower() for token in doc if not token.is_punct and token.text != ',']
    filtered_tokens = [token for token in filtered_tokens if re.match("^[a-zA-Z]+$", token)]
    return filtered_tokens

In [85]:
preprocessed_text = preprocess(text_list)
print(type(preprocessed_text))


Radiohead are an English rock band formed in Abingdon, Oxfordshire, in 1985. They comprise Thom Yorke (vocals, guitar, piano, keyboards); brothers Jonny Greenwood (guitar, keyboards, other instruments) and Colin Greenwood (bass); Ed O'Brien (guitar, backing vocals); and Philip Selway (drums, percussion). They have worked with the producer Nigel Godrich and the cover artist Stanley Donwood since 1994. Radiohead's experimental approach is credited with advancing the sound of alternative rock.
Radiohead signed to EMI in 1991 and released their debut album, Pablo Honey, in 1993. Their debut single, "Creep", was a worldwide hit, and their popularity and critical standing rose with The Bends in 1995. Their third album, OK Computer (1997), is acclaimed as a landmark record and one of the greatest albums in popular music, with complex production and themes of modern alienation. Their fourth album, Kid A (2000), marked a dramatic change in style, incorporating influences from electronic music,

In [86]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in preprocessed_text]
tokenized_corpus = [tokens for tokens in tokenized_corpus if tokens]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300")
model.most_similar("radiohead")


[('coldplay', 0.6285437345504761),
 ('r.e.m.', 0.540037214756012),
 ('metallica', 0.5379277467727661),
 ('u2', 0.5235767960548401),
 ('björk', 0.5166013836860657),
 ('frontman', 0.5074184536933899),
 ('aerosmith', 0.5065386891365051),
 ('weezer', 0.49893084168434143),
 ('wilco', 0.4829169809818268),
 ('bandmates', 0.4812430441379547)]

In [88]:
from collections import Counter

# Flatten the tokenized_corpus
flattened_corpus = [word for sentence in tokenized_corpus for word in sentence]

# Filter tokens that exist in the model's vocabulary
filtered_tokens = [word for word in flattened_corpus if word in model.key_to_index]

# Count the occurrences of each word
word_counts = Counter(filtered_tokens)
word_counts

Counter({'the': 540,
         'and': 344,
         'in': 289,
         'of': 227,
         'a': 222,
         'radiohead': 216,
         'to': 158,
         'for': 106,
         'was': 104,
         'their': 95,
         'on': 92,
         'with': 91,
         'as': 81,
         'album': 77,
         'it': 76,
         'by': 72,
         'yorke': 64,
         'released': 63,
         'were': 57,
         'music': 56,
         'they': 52,
         'that': 51,
         'at': 42,
         'greenwood': 41,
         'had': 37,
         'an': 35,
         'band': 35,
         'from': 35,
         'rock': 30,
         'jonny': 30,
         'he': 28,
         'first': 26,
         'his': 26,
         'is': 24,
         'one': 24,
         'albums': 24,
         'kid': 24,
         'best': 24,
         'all': 24,
         'new': 23,
         'number': 22,
         'not': 21,
         'said': 21,
         'work': 20,
         'have': 19,
         'recorded': 19,
         'songs': 19,
         'i

In [89]:
word_to_index = {}
index_to_word = {}
for idx, word in enumerate(sorted(set(filtered_tokens))):
    word_to_index[word] = idx + 1  # start indexing from 1 (reserve 0 for padding)
    index_to_word[idx + 1] = word

In [90]:
vocab_size = len(word_to_index) + 1 # for padding token at index 0
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size,embedding_dim))

for word, idx in word_to_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6,size = (embedding_dim))

In [91]:
input_seqs, output_words = [], []

for i in range(len(flattened_corpus) - 3):  
    input_seq = flattened_corpus[i:i+3]
    output_word = flattened_corpus[i+3]

    # Check if all words in input_seq and output_word exist in word_to_index
    if all(word in word_to_index for word in input_seq + [output_word]):
        input_seqs.append([word_to_index[word] for word in input_seq])
        output_words.append(word_to_index[output_word])

In [94]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Pad the sequences
X = pad_sequences(input_seqs, maxlen=3, padding='pre')
y= np.array(output_words)


In [95]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.callbacks import EarlyStopping

In [96]:
nn = Sequential()

#"Embedding: Maps words to dense vectors in lower-dimensional space.
nn.add(Embedding(input_dim=vocab_size,
                    output_dim=300,  # Match the second dimension of embedding_matrix
                    weights=[embedding_matrix],
                    input_length=3,
                    trainable=False))

#"LSTM: Recurrent neural network for modeling sequential data effectively.
nn.add(LSTM(1000,return_sequences=True))
nn.add(LSTM(1000))

#Dense: Fully connected layer in neural networks for feature transformation
nn.add(Dense(1000,activation="relu"))

# output layer
nn.add(Dense(vocab_size,activation="softmax"))

In [97]:
nn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │       584,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 584,400 (2.23 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 584,400 (2.23 MB)

In [98]:
early_stop = EarlyStopping(monitor="loss",patience=3)


In [99]:
nn.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [100]:
from tensorflow.keras.utils import to_categorical

# Convert integer targets to one-hot encoding
y_categorical = to_categorical(y, num_classes=vocab_size)


# Fit the model
hist = nn.fit(X, y_categorical, epochs=30, callbacks=[early_stop])


Epoch 1/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 36s 107ms/step - accuracy: 0.0525 - loss: 6.6077
Epoch 2/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 105ms/step - accuracy: 0.0660 - loss: 6.1055
Epoch 3/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 103ms/step - accuracy: 0.0739 - loss: 5.9332
Epoch 4/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 105ms/step - accuracy: 0.1109 - loss: 5.5722
Epoch 5/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 29s 112ms/step - accuracy: 0.1382 - loss: 5.1215
Epoch 6/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 28s 110ms/step - accuracy: 0.1459 - loss: 4.8256
Epoch 7/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 28s 110ms/step - accuracy: 0.1957 - loss: 4.2290
Epoch 8/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 28s 107ms/step - accuracy: 0.2358 - loss: 3.6943
Epoch 9/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 105ms/step - accuracy: 0.3091 - loss: 3.0382
Epoch 10/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 104ms/step - accuracy: 0.4228 - loss: 2.3454
Epoch 11/30
257/257 ━━━━━━━━━━━━━━━━━━━━ 27s 104ms/step - accuracy: 0.5993 - loss: 1.5966
Epoch 12/30
257/257

In [101]:
nn.save("LSTM_with_word2Vec.h5")

In [102]:
nn.evaluate(X,y_categorical)

257/257 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9272 - loss: 0.2070


[0.20781239867210388, 0.9309924244880676]

In [103]:
input_seqs[2]

[74, 567, 1474]

In [104]:
words = [index_to_word[idx] for idx in [1181, 117, 551]]
print(words)

['number', 'artists', 'elements']


In [114]:
def predict_next_word(input_text, model, word_to_index, index_to_word, max_sequence_length=3):
    """Predict the next word given input text."""
    # Tokenize input text
    words = input_text.lower().split()
    
    # Take the last 3 words if there are more than 3
    if len(words) > max_sequence_length:
        words = words[-max_sequence_length:]
    
    # Convert words to indices
    indices = []
    for word in words:
        if word in word_to_index:
            indices.append(word_to_index[word])
        else:
            # Handle unknown words
            print(f"Warning: '{word}' not in vocabulary, skipping.")
    
    # If we don't have any valid indices, return early
    if not indices:
        return "No valid words found in input."
    
    # Ensure we have exactly max_sequence_length indices (pad if necessary)
    if len(indices) < max_sequence_length:
        indices = [0] * (max_sequence_length - len(indices)) + indices
    
    # Reshape for model input (adding batch dimension)
    X_pred = np.array([indices])
    
    # Make prediction
    prediction = model.predict(X_pred, verbose=0)[0]
    
    # Get the index with highest probability
    predicted_index = np.argmax(prediction)
    
    # Convert back to word
    if predicted_index in index_to_word:
        return index_to_word[predicted_index]
    else:
        return "Unknown prediction"

def complete_sentence(input_text, model, word_to_index, index_to_word, num_words=5):
    """Generate a sentence continuation by predicting multiple words."""
    current_text = input_text.lower()
    for _ in range(num_words):
        next_word = predict_next_word(current_text, model, word_to_index, index_to_word)
        current_text += " " + next_word
    
    return current_text

In [115]:
# Example usage
start_text = "Radiohead are an"
completed = complete_sentence(start_text, nn, word_to_index, index_to_word, num_words=5)
print(completed)

radiohead are an english rock band formed in
